In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# default_exp eda

# 事件抽取
http://challenge.xfyun.cn/topic/info?type=hotspot

https://zhuanlan.zhihu.com/p/150190165

## 数据说明
1. 初赛数据说明：

本次比赛初赛为参赛选手提供了6958条中文句子，及其9644条提取结果（存在一对多的情况）：

1.1训练集：共5758条句子，包含句子中对应的触发词、论元等，用于竞赛模型训练。

1.2测试集：共1200条句子。

2. 复赛数据说明：

本次比赛复赛为参赛选手提供了3335条中文句子，及其3384条提取结果（存在一对多的情况）：

2.1训练集：共2456条句子，包含句子中对应的触发词、论元及其角色、事件属性等，用于竞赛模型训练。

2.2测试集：共879条句子。
## 评价指标

本模型依据提交的结果文件，采用F值进行评价。

### 事件触发词及论元抽取评价指标 
对于事件触发词及论元抽取，使用触发词进行事件对齐，对于触发词匹配的事件，计算论元F值。最终F值包括论元与触发词（将触发词当做一种论元计算得分）。论元F值为严格F值与松弛F值的平均得分。

3.1.1 严格F值：预测论元与标注论元必须完全匹配（类型必须正确）

论元准确率Pspan = 预测论元和标注论元匹配的个数 / 预测论元个数

论元召回率Rspan = 预测论元和标注论元匹配的个数 / 标注论元个数

论元F值 F1span = 2 Pspan Rspan /( Pspan + Rspan)

3.1.2 松弛F值：预测论元与标注论元存在字符级别匹配也能得到部分分数（类型必须正确）

论元准确率Pchar = 预测论元和标注论元匹配的字符数 / 预测论元字符数

论元召回率Rchar = 预测论元和标注论元匹配的字符数 / 标注论元字符数

论元F值 F1char = 2 Pchar Rchar /( Pchar + Rchar)

3.1.3 最终得分：F1 = (F1span + F1char) / 2

### 事件属性抽取评价指标

对于事件属性抽取，使用F值进行评价。

属性准确率P = 预测属性和标注属性匹配的个数/ 预测属性个数

属性召回率R = 预测属性和标注属性匹配的个数 / 标注属性个数

属性F值 F1= 2 P R /( P+ R)

# 解题思路
## mine
因为给的数据量很少，只能用预训练模型，finetune。当然预训练模型也不用很复杂。

很好的图像练手的比赛！
## 大佬


# lib导入

In [1]:
# export
import os
import glob
from code.config import * 
from loguru import logger
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行

from sklearn.metrics import classification_report, accuracy_score
import sys
sys.path.append('..')
from mylib.utils.pickle import PickleWrapper

In [2]:

import seaborn as sns
sns.set(font='Arial Unicode MS')  # 解决Seaborn中文显示问题
from mylib.data.processing.eda import describe

In [3]:
args.DATA_DIR

'../../data/contest/20kdxf_event/'

In [4]:
!mkdir {args.DATA_DIR}

In [6]:
!open {args.DATA_DIR}

In [7]:
!ls {args.DATA_DIR}

test
train
事件抽取挑战赛_事件抽取挑战赛提交样例.csv


# EDA

In [126]:
train_raw = pd.read_csv(args.DATA_DIR+'train/train.csv')

In [131]:
tmp = train_raw.time[train_raw.time.notnull()]

## 脏数据

### time 脏数据

邓小平 交通运输部 农业农村部 颐年堂之后 资委会领导人孙越崎

In [137]:
train_raw.loc[train_raw.time=='邓小平', 'object'] = '邓小平'

train_raw.loc[train_raw.time=='邓小平', 'time'] = None

In [144]:
train_raw.loc[train_raw.time=='交通运输部', 'object'] = '交通运输部'

train_raw.loc[train_raw.time=='交通运输部', 'time'] = None

In [146]:
train_raw.loc[train_raw.time=='农业农村部', 'object'] = '农业农村部'

train_raw.loc[train_raw.time=='农业农村部', 'time'] = None

In [151]:
train_raw.loc[train_raw.time=='资委会领导人孙越崎', 'object'] = '资委会领导人孙越崎'

train_raw.loc[train_raw.time=='资委会领导人孙越崎', 'time'] = '6月'

In [149]:
train_raw.loc[train_raw.time=='颐年堂之后', 'time'] = None

### 还有一些奇怪的编码方式 需要确认一下模型的识别效果

１９６７年２月１５日

In [154]:
strange_dict = {
    '１': '1',
    '９': '9',
    '６': '6',
    '７': '7',
    '２': '2',
    '５': '5',
    '３': '3',
    '４': '4',
    '８': '8',
    '０': '0',
}

In [135]:
tmp[~tmp.str.contains(r'[1-9]|[一二三十]')]

74               今年
222              凌晨
228              凌晨
526              今年
601           １８日下午
620            春节期间
822            本世纪初
842             去年起
879            除夕深夜
880            除夕深夜
1160             春节
1276          农业农村部
1302             今年
1303             今年
1304             今年
1305             今年
1408            今年初
1416            近两年
1430             昨天
1441           这个春节
1488             春秋
1663             除夕
1678             除夕
1778             今年
2023             去年
2041          今年上半年
2044            今年初
2077          交通运输部
2130           今年年初
2132             今年
2393            除夕夜
2804            周日晚
2812           今天下午
3034           今天下午
3389             明末
3390             明末
3540             今年
3789        清明节假日期间
3916             次日
3945           两个多月
4023           今天下午
4071             次日
4073             次日
4074             次日
4075             次日
4076             次日
4077             次日
4325          １９３０年
4326          １９３０年
4327          １９３０年


In [156]:
r = str.maketrans(strange_dict)
r

{65297: '1',
 65305: '9',
 65302: '6',
 65303: '7',
 65298: '2',
 65301: '5',
 65299: '3',
 65300: '4',
 65304: '8',
 65296: '0'}

In [157]:
train_raw['news'] = train_raw['news'].map(lambda s: s.translate(r))

In [159]:
train_raw['time'] = train_raw['time'].map(lambda s: s.translate(r), na_action='ignore')

In [163]:
train_raw['trigger'] = train_raw['trigger'].map(lambda s: s.translate(r), na_action='ignore')
train_raw['object'] = train_raw['object'].map(lambda s: s.translate(r), na_action='ignore')
train_raw['subject'] = train_raw['subject'].map(lambda s: s.translate(r), na_action='ignore')
train_raw['location'] = train_raw['location'].map(lambda s: s.translate(r), na_action='ignore')

In [161]:
train_raw['time'].loc[4331]

'1932年'

In [152]:
train_raw.head()

,id,news,trigger,object,subject,time,location
0,1,新华社北京4月7日电（记者王优玲）人力资源社会保障部7日启动第七届大中城市联合招聘高校毕业生...,启动,人力资源社会保障部,第七届大中城市联合招聘高校毕业生春季专场活动,7日,NaN
1,2,活动针对高校毕业生特点和需求，集中归集发布一批适合高校毕业生的就业岗位，并鼓励引导供需双方应...,发布,活动,一批适合高校毕业生的就业岗位,NaN,NaN
2,2,活动针对高校毕业生特点和需求，集中归集发布一批适合高校毕业生的就业岗位，并鼓励引导供需双方应...,鼓励,活动,供需双方,NaN,NaN
3,4,图集4月8日，三大运营商中国移动、中国电信和中国联通将联合举行线上发布会，发布5G消息白皮书...,举行,三大运营商中国移动、中国电信和中国联通,线上发布会,4月8日,NaN
4,4,图集4月8日，三大运营商中国移动、中国电信和中国联通将联合举行线上发布会，发布5G消息白皮书...,发布,三大运营商中国移动、中国电信和中国联通,5G消息白皮书,4月8日,NaN


In [166]:
train_raw.iloc[4331]

id                                                       4583
news        三、林彪死党李作鹏其人李作鹏，1914年出生于江西吉安县，1930年参加中国工农红军，193...
trigger                                                    转入
object                                                    李作鹏
subject                                               中国共产党党员
time                                                    1932年
location                                                  NaN
Name: 4331, dtype: object

In [164]:
train_raw.to_pickle('train_raw.pkl')

In [44]:
'新华社北京4月7日电（记者王优玲）人力资源'.find('北京的')

-1

In [54]:
train_raw = train_raw.fillna('')

In [76]:
marks_list = []
for line in train_raw.itertuples():
    marks = ['O'] * len(line.news)
    for e, obj in enumerate('trigger object subject time location'.split()):
#         print(line[e+3])
        if line[e+3] == '': continue 
        start = line.news.find(line[e+3])
        if start != -1:
            marks[start] = f'B-{obj}'
            for i in range(len(line[e+3])-1):
                marks[start+i+1] = f'I-{obj}'
    marks_list.append(marks)
#     print(marks)
#     break

In [58]:
len(marks_list)

8000

In [60]:
news_list = train_raw.news.map(list).tolist()

In [77]:
results = []
for news, marks in zip(news_list, marks_list):
    ll = []
    for n, m in zip(news, marks):
        ll.append(f'{n} {m}')
    results.append('\n'.join(ll))

In [68]:
len(results)

8000

In [67]:
with open('kdxf_event.txt', 'w', encoding='utf8') as f:
    f.write('\n\n'.join(results))

In [78]:
with open('train.txt', 'w', encoding='utf8') as f:
    f.write('\n\n'.join(results[:7000]))

In [79]:
with open('dev.txt', 'w', encoding='utf8') as f:
    f.write('\n\n'.join(results[7000:]))

In [61]:
news_list[0]

['新',
 '华',
 '社',
 '北',
 '京',
 '4',
 '月',
 '7',
 '日',
 '电',
 '（',
 '记',
 '者',
 '王',
 '优',
 '玲',
 '）',
 '人',
 '力',
 '资',
 '源',
 '社',
 '会',
 '保',
 '障',
 '部',
 '7',
 '日',
 '启',
 '动',
 '第',
 '七',
 '届',
 '大',
 '中',
 '城',
 '市',
 '联',
 '合',
 '招',
 '聘',
 '高',
 '校',
 '毕',
 '业',
 '生',
 '春',
 '季',
 '专',
 '场',
 '活',
 '动']

In [59]:
marks_list[2]

['B-object',
 'I-object',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'B-trigger',
 'I-trigger',
 'o',
 'o',
 'B-subject',
 'I-subject',
 'I-subject',
 'I-subject',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o',
 'o']

In [32]:
sub_raw = pd.read_csv(args.DATA_DIR+'事件抽取挑战赛_事件抽取挑战赛提交样例.csv')

In [33]:
sub_raw.head()

,id,trigger,object,subject,time,location
0,7953,恢复,深航,湖北其他城市的航班运力,NaN,NaN
1,7955,恢复,深航,湖北其他城市的航班运力,NaN,NaN
2,7956,恢复,深航,湖北其他城市的航班运力,NaN,NaN
3,7956,恢复,深航,湖北其他城市的航班运力,NaN,NaN
4,7959,恢复,深航,湖北其他城市的航班运力,NaN,NaN


In [71]:
test_raw = pd.read_csv(args.DATA_DIR+'test/test.csv')

In [72]:
test_raw.head()

,id,news
0,7953,此外，深航积极恢复湖北其他城市的航班运力，已于近日陆续恢复襄阳往返深圳、惠州、沈阳、珠海，宜...
1,7955,同时，因其违反中国法律、法规规定，妨害社会管理，根据《出境入境管理法》第八十一条第一款，越秀...
2,7956,2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...
3,7959,广州男子涉嫌偷拍女同学发布淫秽信息，学校发声4月8日9时许，广州警方发布通报，经过缜密调查，...
4,7960,作为广东深圳一家节能企业的董事长，从1月29日开始，李海建便来到武汉参与医疗废弃物焚烧车间建设


In [99]:
test_raw.shape

(1200, 2)

In [73]:
test = []
for i in test_raw.news.map(list).tolist():
    test.append('\n'.join(i))

In [75]:
with open('test.txt', 'w', encoding='utf8') as f:
    f.write('\n\n'.join(test))

## gen预测结果

In [124]:
with open('test_predictions_4_342.txt', 'r', encoding='utf8') as f:
    lines = f.readlines()

marks = [line.strip().split()[-1] if line.strip() else '##' for line in lines]



marks = ' '.join(marks)

marks = marks.split(' ## ')

print(len(marks))

preds = []
for ii, marks_ in enumerate(marks):
#     print(marks_)
#     print(test_raw.iloc[ii]['news'])
    d = {
        'trigger': None,
        'object': None,
        'time': None,
        'location': None,
    }
    start = -1
    range_ = 1
    label = None
    for i, mark in enumerate(marks_.split()):
        d['id'] = test_raw.iloc[ii]['id']
        if mark.startswith('B'):
            if label and start != -1:
                d[label] = test_raw.iloc[ii]['news'][start: start+range_]
            start = i
            range_=1
            label = mark.split('-')[-1]
        elif mark.startswith('I'):
            if label == mark.split('-')[-1]:
                range_ += 1
            elif label:
                d[label] = test_raw.iloc[ii]['news'][start: start+range_]
                start = -1
                range_ = 1
                label = None
        elif mark.startswith('O'):
            if label:
                d[label] = test_raw.iloc[ii]['news'][start: start+range_]
                start = -1
                range_ = 1
                label = None
    preds.append(d)
#     print(d)
#         break
#     break

sub_ = pd.DataFrame(preds)

test_raw.head()

sub_[sub_raw.columns].to_csv(args.DATA_DIR+'sub_0804_342.csv', index=False)

1200


In [125]:
sub_.sample(20)

,trigger,object,time,location,id,subject
995,发生,马达加斯加,2017年,None,9668,洪涝灾害
216,发布,黑龙江省政府办公厅,None,None,8354,《关于坚决禁止猎捕林蛙加强野生动物管理的紧急通知》
481,提,中国人,六十多年前,None,8894,NaN
189,形成,自治区质量技术监督局,None,None,8310,《规模化银星竹鼠场建设规范》《银星竹鼠饲养技术规范》等地方标准
53,None,西班牙,None,None,8049,NaN
75,加,费莱尼,2,None,8087,NaN
650,启动,海南省,2016年,None,9161,NaN
218,召开,东北林业大学野生动物资源学院,2018年11月18日,None,8356,一个座谈会
1115,发生,深圳市龙岗区横岗一路段,2013年5月,None,9851,塌陷
1066,授牌,赵长青,None,None,9778,NaN


In [13]:
train_raw.news.str.len().describe()

count    8000.000000
mean       71.780875
std        39.728918
min        10.000000
25%        45.000000
50%        63.000000
75%        88.000000
max       397.000000
Name: news, dtype: float64

In [14]:
test_raw.news.str.len().describe()

count    1200.000000
mean       62.677500
std        36.050009
min        10.000000
25%        38.000000
50%        54.000000
75%        78.000000
max       342.000000
Name: news, dtype: float64

In [36]:
train_raw.trigger.nunique()

1256

In [37]:
train_raw['trigger'].value_counts()>1

召开       True
发布       True
参加       True
接受       True
发表       True
        ...  
突发      False
稿酬      False
提前      False
赶回      False
没有遇到    False
Name: trigger, Length: 1256, dtype: bool

In [38]:
lb = (train_raw['trigger'].value_counts()>1).reset_index()
lb.columns = ['lb','flag']
lb = list(lb[lb['flag']==True].lb.values)
predicate2id = {j:i for i,j in enumerate(lb)}
id2predicate = {i:j for i,j in enumerate(lb)}

In [39]:
len(lb)

478

In [29]:
lb[:5]

['召开', '发布', '参加', '接受', '发表']

In [40]:
predicate2id

{'召开': 0,
 '发布': 1,
 '参加': 2,
 '接受': 3,
 '发表': 4,
 '举行': 5,
 '开展': 6,
 '提出': 7,
 '成立': 8,
 '推出': 9,
 '作出': 10,
 '发生': 11,
 '公布': 12,
 '到达': 13,
 '离开': 14,
 '建立': 15,
 '发现': 16,
 '组织': 17,
 '参与': 18,
 '恢复': 19,
 '启动': 20,
 '支持': 21,
 '找到': 22,
 '印发': 23,
 '出任': 24,
 '介绍': 25,
 '出席': 26,
 '制定': 27,
 '提供': 28,
 '增加': 29,
 '转入': 30,
 '设立': 31,
 '通报': 32,
 '宣布': 33,
 '举办': 34,
 '反对': 35,
 '指出': 36,
 '导致': 37,
 '形成': 38,
 '选择': 39,
 '批准': 40,
 '做出': 41,
 '购买': 42,
 '推动': 43,
 '推进': 44,
 '执行': 45,
 '调查': 46,
 '申请': 47,
 '关注': 48,
 '加入': 49,
 '分享': 50,
 '投入': 51,
 '给予': 52,
 '减少': 53,
 '解决': 54,
 '控制': 55,
 '制造': 56,
 '被确诊': 57,
 '检查': 58,
 '披露': 59,
 '打造': 60,
 '开启': 61,
 '会见': 62,
 '报道': 63,
 '改变': 64,
 '分析': 65,
 '进行': 66,
 '创造': 67,
 '调整': 68,
 '通过': 69,
 '带来': 70,
 '处理': 71,
 '影响': 72,
 '担任': 73,
 '建议': 74,
 '开发': 75,
 '告诉': 76,
 '提高': 77,
 '学习': 78,
 '发出': 79,
 '到': 80,
 '开放': 81,
 '前往': 82,
 '完成': 83,
 '抵达': 84,
 '建设': 85,
 '乘': 86,
 '实施': 87,
 '建成': 88,
 '发行': 89,
 '来到': 90,
 '解除': 91,

In [30]:
train = train_raw
train = train[train['trigger'].isin(lb)]
train['label_1'] = list(map(lambda x,y,z:(str(x),str(y),str(z)),train['object'],train['trigger'],train['subject']))
train = train.groupby('news')['label_1'].apply(list).reset_index()
train = list(map(lambda x,y:{'text':x,'spo_list':y},train['news'],train['label_1']))

train_data = train[:5000]
valid_data = train[5000:]

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
!cat ../../../Downloads/NER-de-dev.tsv | grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > dev.txt.tmp

In [22]:
!cat ../../../Downloads/NER-de-train.tsv | grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > train.txt.tmp

In [23]:
!cat ../../../Downloads/NER-de-test.tsv | grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > test.txt.tmp

In [24]:
!cat ../../../Downloads/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [16]:
!python3 scripts/preprocess.py dev.txt.tmp $BERT_MODEL $MAX_LENGTH > dev.txt

python3: can't open file 'scripts/preprocess.py': [Errno 2] No such file or directory


In [19]:
import sys

from transformers import AutoTokenizer


dataset = 'dev.txt.tmp'
model_name_or_path = 'bert-base-multilingual-cased'
max_len = 128

subword_len_counter = 0

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
max_len -= tokenizer.num_special_tokens_to_add()
lines = []
with open(dataset, "rt") as f_p:
    for line in f_p:
        line = line.rstrip()

        if not line:
            print(line)
            subword_len_counter = 0
            continue

        token = line.split()[0]

        current_subwords_len = len(tokenizer.tokenize(token))

        # Token contains strange control characters like \x96 or \x95
        # Just filter out the complete line
        if current_subwords_len == 0:
            continue

        if (subword_len_counter + current_subwords_len) > max_len:
            print("")
            print(line)
            subword_len_counter = current_subwords_len
            continue

        subword_len_counter += current_subwords_len

#         print(line)
        lines.append(line)

I0803 16:24:24.828222 140735683666816 file_utils.py:39] PyTorch version 1.5.1 available.
I0803 16:24:28.761773 140735683666816 file_utils.py:55] TensorFlow version 2.2.0 available.


ImportError: dlopen(/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tokenizers/tokenizers.cpython-37m-darwin.so, 2): Symbol not found: ____chkstk_darwin
  Referenced from: /Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tokenizers/tokenizers.cpython-37m-darwin.so (which was built for Mac OS X 10.15)
  Expected in: /usr/lib/libSystem.B.dylib
 in /Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tokenizers/tokenizers.cpython-37m-darwin.so